In [20]:
import os, shutil
import sys
import h5py
import numpy as np
import cv2
from PIL import Image
sys.path.insert(0, './lib_keras/')
from help_functions import write_hdf5
import configparser

In [21]:


config = configparser.RawConfigParser()
config.read('pre_configuration.txt')
mode = config.get('setting', 'mode')
original_img_train_path = config.get('path','original')
ground_truth_img_train_path = config.get('path','ground')
border_masks_imgs_train_path = config.get('path','mask')
save_dir_path = config.get('path', 'save_path')
save_dir_path = save_dir_path + mode + '/'

print('mode : {} \ntrain img : {} \nlabel img : {} \nsave path : {}'.format(mode, original_img_train_path,\
                                                                            ground_truth_img_train_path, save_dir_path))


if os.path.isdir('./'+save_dir_path) == False:
    os.mkdir('./'+save_dir_path)
else:
    print('already exist the folder in this path : {}'.format('./'+save_dir_path))

mode : inha_oct 
train img : ./data/inha_train/training/train/ 
label img : ./data/inha_train/training/label/ 
save path : ./data/hdf5_data/inha_oct/
already exist the folder in this path : ././data/hdf5_data/inha_oct/


In [22]:
if mode == 'inha_oct' :
    path, dirs, files = next(os.walk(original_img_train_path))
    num_imgs = len(files)
    img = cv2.imread(original_img_train_path + files[0])
    img_shape = np.shape(img)
    
    if len(img_shape) == 3:
        #ch = 3 or 1
        channels = img_shape[2]
        img_height = img_shape[0]
        img_width = img_shape[1]
        
    elif len(img_shape) == 2:
        # ch = none
        img_height = img_shape[0]
        img_width = img_shape[1]
    
    print('img shape : {} \nnumber of imgs : {} \ndirs : {}\n'.format(img_shape, num_imgs, files))

img shape : (500, 760, 3) 
number of imgs : 40 
dirs : ['raw_data_31.png', 'raw_data_34.png', 'raw_data_29.png', 'raw_data_04.png', 'raw_data_18.png', 'raw_data_20.png', 'raw_data_00.png', 'raw_data_13.png', 'raw_data_14.png', 'raw_data_03.png', 'raw_data_09.png', 'raw_data_38.png', 'raw_data_10.png', 'raw_data_19.png', 'raw_data_12.png', 'raw_data_05.png', 'raw_data_08.png', 'raw_data_01.png', 'raw_data_37.png', 'raw_data_30.png', 'raw_data_22.png', 'raw_data_36.png', 'raw_data_16.png', 'raw_data_24.png', 'raw_data_07.png', 'raw_data_15.png', 'raw_data_23.png', 'raw_data_17.png', 'raw_data_25.png', 'raw_data_35.png', 'raw_data_39.png', 'raw_data_27.png', 'raw_data_28.png', 'raw_data_21.png', 'raw_data_06.png', 'raw_data_32.png', 'raw_data_26.png', 'raw_data_11.png', 'raw_data_02.png', 'raw_data_33.png']



In [29]:
def get_datasets(imgs_dir,groundTruth_dir,borderMasks_dir = False ,train_test="null"):
    imgs = np.empty((num_imgs,img_height,img_width,channels))
    groundTruth = np.empty((num_imgs,img_height,img_width,channels))
    
    print('img dir : ',imgs_dir)
    for count, filename in enumerate(sorted(os.listdir(imgs_dir)), start=0):
        print ("original image: " +filename)
        img = Image.open(imgs_dir+filename)
        imgs[count] = np.asarray(img)
        print('file name : ',imgs_dir+ filename)
    
    print('ground truth dir : ', groundTruth_dir)
    for count, filename in enumerate(sorted(os.listdir(groundTruth_dir)), start=0):
        groundTruth_name = filename
        print ("ground truth name: " + groundTruth_name)
        g_truth = Image.open(groundTruth_dir + groundTruth_name)
        if len(np.shape(g_truth)) !=2:
            g_truth = cv2.imread(groundTruth_dir+groundTruth_name)
            #g_truth = g_truth[:,:,0]
        groundTruth[count] = np.asarray(g_truth)
        
    if borderMasks_dir != False:
        border_masks = np.empty((num_imgs,img_height,img_width))
        for count, filename in enumerate(sorted(os.listdir(borderMasks_dir)), start=0):
            border_masks_name = ""
            if train_test=="train":
                border_masks_name = filename
            elif train_test=="test":
                border_masks_name = filename
            else:
                print ("specify if train or test!!")
                exit()
            print ("border masks name: " + border_masks_name)
            b_mask = Image.open(borderMasks_dir + border_masks_name)
            b_mask = np.asarray(b_mask)
            if len(np.shape(b_mask)) ==3:
                b_mask = b_mask[:,:,0]
            
            border_masks[count] = b_mask
            #print(b_mask,'\n')
    
        print(border_masks.shape)

        print ("imgs max: " +str(np.max(imgs)))
        print ("imgs min: " +str(np.min(imgs)))
        assert(np.max(groundTruth)==255 and np.max(border_masks)==255)
        assert(np.min(groundTruth)==0 and np.min(border_masks)==0)
        print ("ground truth and border masks are correctly withih pixel value range 0-255 (black-white)")

        #reshaping for my standard tensors
        imgs = np.transpose(imgs,(0,3,1,2))
        assert(imgs.shape == (num_imgs,channels,img_height,img_width))
        groundTruth = np.reshape(groundTruth,(num_imgs,channels,img_height,img_width))
        border_masks = np.reshape(border_masks,(num_imgs,1,img_height,img_width))
        assert(groundTruth.shape == (num_imgs,channels,img_height,img_width))
        assert(border_masks.shape == (num_imgs,1,img_height,img_width))
        
        return imgs, groundTruth, border_masks
        
    else:
        print ("imgs max: " +str(np.max(imgs)))
        print ("imgs min: " +str(np.min(imgs)))
        assert(np.max(groundTruth)==255)
        assert(np.min(groundTruth)==0)
        print ("ground truth and border masks are correctly withih pixel value range 0-255 (black-white)")
        #reshaping for my standard tensors
        imgs = np.transpose(imgs,(0,3,1,2))
        assert(imgs.shape == (num_imgs,channels,img_height,img_width))
        groundTruth = np.reshape(groundTruth,(num_imgs,channels,img_height,img_width))
        assert(groundTruth.shape == (num_imgs,channels,img_height,img_width))

        return imgs, groundTruth


In [30]:
if mode =='inha_oct':
    imgs_train,groundTruth_train =get_datasets(original_img_train_path,ground_truth_img_train_path,train_test='train')
    write_hdf5(imgs_train, save_dir_path + mode+"_train.hdf5")
    write_hdf5(groundTruth_train, save_dir_path+mode + "_groundTruth_train.hdf5")
    #write_hdf5(border_masks_train,dataset_dir_path+mode + "_borderMasks_train.hdf5")

img dir :  ./data/inha_train/training/train/
original image: raw_data_00.png
file name :  ./data/inha_train/training/train/raw_data_00.png
original image: raw_data_01.png
file name :  ./data/inha_train/training/train/raw_data_01.png
original image: raw_data_02.png
file name :  ./data/inha_train/training/train/raw_data_02.png
original image: raw_data_03.png
file name :  ./data/inha_train/training/train/raw_data_03.png
original image: raw_data_04.png
file name :  ./data/inha_train/training/train/raw_data_04.png
original image: raw_data_05.png
file name :  ./data/inha_train/training/train/raw_data_05.png
original image: raw_data_06.png
file name :  ./data/inha_train/training/train/raw_data_06.png
original image: raw_data_07.png
file name :  ./data/inha_train/training/train/raw_data_07.png
original image: raw_data_08.png
file name :  ./data/inha_train/training/train/raw_data_08.png
original image: raw_data_09.png
file name :  ./data/inha_train/training/train/raw_data_09.png
original image: